In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
import pandas as pd
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

In [3]:
weather_add_1 = pd.read_csv('weather_holiday_metro_1.csv',encoding='CP949')
#weather_add_1
weather_add_1['일시'] = pd.to_datetime(weather_add_1['일시'], format='%Y-%m-%d', errors='raise')
weather_add_1['휴일여부'] = weather_add_1['일시'].dt.weekday       # 요일숫자(0-월, 1-화) 
weather_add_1.loc[(weather_add_1['휴일여부'] <= 3, '휴일여부')] = 0
weather_add_1.loc[(weather_add_1['휴일여부'] == 6, '휴일여부')] = 0
weather_add_1.loc[(weather_add_1['휴일여부'] > 3, '휴일여부')] = 1
weather_add_1.head(15)

,일시,평균기온(°C),일강수량(mm),평균 풍속(m/s),노선명,역명,승하차인원,휴일여부
0,2019-01-01,-3.7,0.0,1.8,2호선,1,75273,0
1,2019-01-02,-3.5,0.0,1.5,2호선,1,233032,0
2,2019-01-03,-1.9,0.0,1.1,2호선,1,245143,0
3,2019-01-04,-0.5,0.0,1.3,2호선,1,264826,1
4,2019-01-05,-1.5,0.0,2.0,2호선,1,185844,1
5,2019-01-06,-1.6,0.0,1.3,2호선,1,103482,0
6,2019-01-07,-0.7,0.0,1.4,2호선,1,242619,0
7,2019-01-08,-2.3,0.0,2.0,2호선,1,246658,0
8,2019-01-09,-3.6,0.0,1.3,2호선,1,246022,0
9,2019-01-10,0.0,0.0,1.3,2호선,1,249196,0


In [4]:
#코로나 확진자 데이터 불러오기
covid = pd.read_excel('covid.xlsx')
covid['date'] = pd.to_datetime(covid['date'], format='%Y%m%d', errors='raise')
covid.rename(columns={'date':'일시'},inplace=True)
covid

,일시,confirmed
0,2020-01-21,1
1,2020-01-22,0
2,2020-01-23,0
3,2020-01-24,1
4,2020-01-25,0
...,...,...
114,2020-05-14,29
115,2020-05-15,27
116,2020-05-16,19
117,2020-05-17,13


In [5]:
#코로나 확진자 데이터 합치기 (코로나 이전 시기인 널값은 0으로 처리)
weather_add_covid = pd.merge(left = weather_add_1,
        right = covid,
        how = 'left',
        left_on = '일시',
        right_on = '일시')
weather_add_covid = weather_add_covid.fillna(0)
weather_add_covid

,일시,평균기온(°C),일강수량(mm),평균 풍속(m/s),노선명,역명,승하차인원,휴일여부,confirmed
0,2019-01-01,-3.7,0.0,1.8,2호선,1,75273,0,0.0
1,2019-01-02,-3.5,0.0,1.5,2호선,1,233032,0,0.0
2,2019-01-03,-1.9,0.0,1.1,2호선,1,245143,0,0.0
3,2019-01-04,-0.5,0.0,1.3,2호선,1,264826,1,0.0
4,2019-01-05,-1.5,0.0,2.0,2호선,1,185844,1,0.0
...,...,...,...,...,...,...,...,...,...
4027,2020-05-14,20.9,0.0,0.4,5호선,8,19958,0,29.0
4028,2020-05-15,16.9,9.0,0.8,5호선,8,13201,1,27.0
4029,2020-05-16,17.0,1.5,0.6,5호선,8,11312,1,19.0
4030,2020-05-17,19.2,0.0,0.7,5호선,8,14847,0,13.0


In [6]:
#결측치 없음
# weather_add_covid.to_csv('wether_metro_covid.csv',encoding='CP949')
weather_add_covid.isna().sum()

일시            0
평균기온(°C)      0
일강수량(mm)      0
평균 풍속(m/s)    0
노선명           0
역명            0
승하차인원         0
휴일여부          0
confirmed     0
dtype: int64

In [0]:
weather_index = weather_add_covid.set_index('일시')
weather_index.drop('노선명',axis=1,inplace=True) 

In [0]:
from sklearn.metrics import make_scorer

def mae(prediction, correct):
    prediction = np.array(prediction)
    correct = np.array(correct)
    
    difference = correct - prediction
    abs_val = abs(difference)
    
    score = abs_val.mean()
    
    return score

mae_scorer = make_scorer(mae)
mae_scorer

make_scorer(mae)

In [0]:
def predict_columns(data_frame, x_cols, y_col):
  for i in data_frame.역명.unique():
    df_num = data_frame[data_frame.역명 == i]
    size = int(len(df_num) * 0.8)
    train = df_num[:size]
    test = df_num[size:]
    
    reg_models = [LinearRegression(), Ridge(), Lasso(), ElasticNet(), GradientBoostingRegressor(), SVR(), XGBRegressor()]
    reg_model_names = ["LinearRegression", "Ridge", "Lasso", "ElasticNet", "GradientBoositng", " SupportVector", "XGBoost"]
    print('역명 {} '.format(i))
    for z in range(len(reg_models)):
        reg = reg_models[z].fit(train[x_cols], train[y_col])
        pred = reg_models[z].predict(test[x_cols])
        print('{} : {}'.format(reg_model_names[z], mae(test[y_col], pred.astype(int))))

In [0]:
predict_columns(weather_index, ['평균기온(°C)', '일강수량(mm)', '평균 풍속(m/s)','휴일여부','confirmed'], '승하차인원')

역명 1 
LinearRegression : 872878.4752475248
Ridge : 860772.5643564357
Lasso : 872284.7623762377
ElasticNet : 219415.52475247523
GradientBoositng : 59458.653465346535
 SupportVector : 80957.52475247525
[05:01:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBoost : 48836.28712871287
역명 2 
LinearRegression : 1002411.0297029703
Ridge : 988684.6534653465
Lasso : 1001871.9702970297
ElasticNet : 247436.9207920792
GradientBoositng : 32862.21782178218
 SupportVector : 48906.66336633663
[05:01:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBoost : 29817.821782178216
역명 3 
LinearRegression : 1150641.7227722772
Ridge : 1134370.3267326732
Lasso : 1150015.297029703
ElasticNet : 282367.2079207921
GradientBoositng : 41143.574257425746
 SupportVector : 70631.64356435643
[05:01:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now de

In [0]:
from sklearn.model_selection import train_test_split

df_num = weather_index[weather_index.역명 == 7]
x = df_num[['평균기온(°C)', '일강수량(mm)', '평균 풍속(m/s)','휴일여부','confirmed']]
y = df_num[['승하차인원']]
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, test_size=0.2)
mlr = LinearRegression()
mlr.fit(x_train, y_train)
train_effect = [[14.1, 14, 1.2, 0, 13]]
my_predict = mlr.predict(train_effect)

In [25]:
my_predict

array([[68692.04944557]])

In [0]:
import statsmodels.api as sm
df_num = weather_index[weather_index.역명 == 2]
y=df_num.승하차인원
x=df_num[['평균기온(°C)', '일강수량(mm)', '평균 풍속(m/s)','휴일여부','confirmed']]
x=sm.add_constant(x)
model=sm.OLS(y,x).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  승하차인원   R-squared:                       0.471
Model:                            OLS   Adj. R-squared:                  0.466
Method:                 Least Squares   F-statistic:                     88.72
Date:                Sun, 07 Jun 2020   Prob (F-statistic):           1.26e-66
Time:                        05:52:24   Log-Likelihood:                -5740.1
No. Observations:                 504   AIC:                         1.149e+04
Df Residuals:                     498   BIC:                         1.152e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.169e+05   4266.153     27.401      0.000    1.09e+05    1.25e+05
평균기온(°C)      48.3108    107.231      0.451      0.653    -162.371     258.992
일강수량(mm)     -42.7012    110.538     -0.386      0.699    -259.880     174.478
평균 풍속(m/s) -8913.3371   2343.974     -3.803      0.000   -1.35e+04   -4308.041
휴일여부         2.87e+04   2123.406     13.516      0.000    2.45e+04    3.29e+04
confirmed   -174.5274     11.316    -15.423      0.000    -196.761    -152.294
==============================================================================
Omnibus:                       42.534   Durbin-Watson:                   0.500
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               51.978
Skew:                          -0.712   Prob(JB):                     5.17e-12
Kurtosis:                       3.670   Cond. No.                         443.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""